<a href="https://colab.research.google.com/github/m4tuuc/RAG_test/blob/main/RAG_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

RAG o también Retrieval-augmented Generation es una técnica que combina la recuperación de información (retrieval) con la generación de texto.

Este enfoque permite a los modelos de lenguaje aprovechar una base de datos externa de textos para complementar y enriquecer sus respuestas, mejorando la relevancia y exactitud de la información generada.

In [1]:
import pandas as pd
import numpy as np
import os

In [ ]:
!pip install --upgrade langchain langchain-core langchain-openai
!pip install -U sentence-transformers

In [2]:
df = pd.read_csv("./data") #cargamos con nuestro set de datos

In [3]:
import re
import unicodedata
df_data = df.astype(str)
def preprocesar_texto(texto):
    texto = re.sub(r"[^a-zA-ZáéíóúÁÉÍÓÚñÑ\s]", "", texto)
    # Normalizar acentos (ej: "é" → "e")
    texto = unicodedata.normalize("NFKD", texto).encode("ASCII", "ignore").decode("utf-8")
    return texto

for columna in df_data.columns:
  df_data[columna] = df_data[columna].apply(preprocesar_texto)

## 1. Diviviendo la informacion en chunks
Los documentos son divididos en pedacitos, o chunks, para que puedan ser correctamente procesados en este contexto.

In [4]:
from langchain.docstore.document import Document

documents = []
for index, row in df_data.iterrows():
  doc = Document(page_content=row['Sinopsis'], metadata={'source': f'row_{index}'})
  documents.append(doc)

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 450,
    chunk_overlap = 50
)
splits = text_splitter.create_documents(df_data['Sinopsis'])

for index, split in enumerate(splits):
  print(f"SPLIT + {index + 1}")
  print("--")

SPLIT + 1
--
SPLIT + 2
--
SPLIT + 3
--
SPLIT + 4
--
SPLIT + 5
--
SPLIT + 6
--
SPLIT + 7
--
SPLIT + 8
--
SPLIT + 9
--
SPLIT + 10
--
SPLIT + 11
--
SPLIT + 12
--
SPLIT + 13
--
SPLIT + 14
--
SPLIT + 15
--
SPLIT + 16
--
SPLIT + 17
--
SPLIT + 18
--
SPLIT + 19
--
SPLIT + 20
--
SPLIT + 21
--
SPLIT + 22
--
SPLIT + 23
--
SPLIT + 24
--
SPLIT + 25
--
SPLIT + 26
--
SPLIT + 27
--
SPLIT + 28
--
SPLIT + 29
--
SPLIT + 30
--
SPLIT + 31
--
SPLIT + 32
--
SPLIT + 33
--
SPLIT + 34
--
SPLIT + 35
--
SPLIT + 36
--
SPLIT + 37
--
SPLIT + 38
--
SPLIT + 39
--
SPLIT + 40
--
SPLIT + 41
--
SPLIT + 42
--
SPLIT + 43
--
SPLIT + 44
--
SPLIT + 45
--
SPLIT + 46
--
SPLIT + 47
--
SPLIT + 48
--
SPLIT + 49
--
SPLIT + 50
--


In [6]:
#Guardamos los chunks

chunks = []
for split in splits:
  chunks.append(split.page_content)

df = pd.DataFrame(chunks, columns=['Text'])

##Generamos los embeddings
Usamos Sentence Transformer para transformar la informacion a representacion numerica

In [7]:
from sentence_transformers import SentenceTransformer
#Bloque de codigo sacado de la documentacion.
# 1. Load a pretrained Sentence Transformer model
model = SentenceTransformer("all-MiniLM-L6-v2")

# The sentences to encode
sentences = df['Text']
# 2. Calculate embeddings by calling model.encode()
embeddings = model.encode(sentences)
print(embeddings.shape)
# [3, 384]

# 3. Calculate the embedding similarities
similarities = model.similarity(embeddings, embeddings)
print(similarities)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


(50, 384)
tensor([[1.0000, 0.3984, 0.4225,  ..., 0.4011, 0.2624, 0.3261],
        [0.3984, 1.0000, 0.1403,  ..., 0.3391, 0.3884, 0.3755],
        [0.4225, 0.1403, 1.0000,  ..., 0.4320, 0.3002, 0.3271],
        ...,
        [0.4011, 0.3391, 0.4320,  ..., 1.0000, 0.4212, 0.5483],
        [0.2624, 0.3884, 0.3002,  ..., 0.4212, 1.0000, 0.4506],
        [0.3261, 0.3755, 0.3271,  ..., 0.5483, 0.4506, 1.0000]])


In [8]:


#Buscamos el chunk que nos interesa
def find_best(query, dataframe, model, embeddings):
  query_embedding = model.encode(query)
  query_embedding = query_embedding.reshape(1, -1)
  dot_product = np.dot(embeddings, query_embedding.T)
  idx = np.argmax(dot_product)
  return dataframe.iloc[idx]['Text']

query = "Hombre solo"
respuesta = find_best(query, df, model, embeddings)
print(respuesta)

Un hombre revive el mismo dia una y otra vez aprendiendo a aprovecharlo al maximo


##Generar la respuesta con un LLM
Use la api de OpenAI, modelos como Llama, Claude tambien sirven.

In [20]:
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI


#from langchain_openai import ChatOpenAI

# Para obtener la API desde Secrets en Google Colab. Si usas otro entorno debes adaptar esto.
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OpenAI')
os.environ["OpenAI"] = OPENAI_API_KEY



llm_model = ChatOpenAI(model_name="gpt-4.1", openai_api_key=OPENAI_API_KEY)


In [21]:
from langchain_core.prompts import ChatPromptTemplate

output_parser = StrOutputParser()


prompt = ChatPromptTemplate.from_messages([
    ("system", """
      Eres un bot servicial e informativo que arma pequeñas historias utilizando el texto referencia incluido a continuación.
      Asegúrate de responder en una oración completa, siendo exhaustivo y proporcionando toda la información de fondo relevante.
      Sin embargo, estás hablando con una audiencia no técnica, por lo que debes desglosar los conceptos complicados y mantener un tono amigable y conversacional.
      Por favor, responde en el idioma de la pregunta.

      PREGUNTA: '{query}'
      TEXTO DE REFERENCIA: '{relevant_passage}'

      RESPUESTA:
    """
    ),
])

chain = prompt | llm_model | output_parser

In [ ]:
chain.invoke({
    "query": query,
    "relevant_passage": best_passage,
    })